# Create Functional Data
Populate a functional attribute with data fitted using the Python `numpy` library.

## Import libraries and define a polynomial fit function
This example populates a functional attribute for *Ultimate Tensile Strength vs Temperature* in one table from a polynomial fit of the individual attributes in another table.

In [1]:
from datetime import datetime
import numpy as np
from GRANTA_MIScriptingToolkit import granta as mpy

def My4degPolyFitFunc(x, a, b, c, d, e):
    return a*np.power(x, 4) + b*np.power(x, 3) + c*np.power(x, 2) + d*x + e

## Specify database and table
The source data will come from the *Tensile Statistical Data* table.

In [2]:
mi = mpy.connect('http://localhost/mi_servicelayer', autologon=True)

db = mi.get_db(db_key='MI_Training')
db.set_unit_system('Metric', absolute_temperatures=True)

table = db.get_table('Tensile Statistical Data')

## Search for test data
Construct a search for records that are appropriately named, and in which both the *Ultimate Tensile Strength* and *Test Temperature* attributes are populated.

In [3]:
search_criteria = [mpy.SearchCriterion(attribute = mpy.PseudoAttributeDefinition('name'), operator='CONTAINS', value='AMS 6520'),
                   table.attributes['Ultimate Tensile Strength'].search_criterion(exists=True),
                   table.attributes['Test Temperature'].search_criterion(exists=True)]

Perform the search and extract the results.

In [4]:
results = table.search_for_records_where(search_criteria)

Extract the attribute values from the returned records into x and y values.

In [5]:
table.bulk_fetch(results, attributes=['Test Temperature', 'Ultimate Tensile Strength'])
x_values = [r.attributes['Test Temperature'].value for r in results]
y_values = [r.attributes['Ultimate Tensile Strength'].value for r in results]

## Fit the test data
Fit a fourth-order polynomial to your x and y data.

In [6]:
coeffs = np.polyfit(x_values, y_values, 4)

Generate x and y values for the fitted equation, using the function you defined at the start.

In [7]:
x_fit = np.linspace(np.amin(x_values), np.amax(x_values), 20)
y_fit = My4degPolyFitFunc(x_fit, *coeffs)

## Create a record to store the data in 
The resulting functional data will be written into the *Design Data* table, using the same unit system.

In [8]:
designdata = db.get_table('Design Data')

Create a new record to store your functional data.

In [9]:
now = datetime.now().strftime("%c")
recordName = 'STK Example 8:{}'.format(now)
record = designdata.create_record(recordName, subsets={'Metals'})
record.color = 'Green'

Access the (empty) functional attribute, and view its column headers.

In [10]:
func = record.attributes['Tens. Ult. Stress (L-dir) with Temp.']
func.column_headers

['Y min (Tens. Ult. Stress (L-dir) with Temp. [MPa])',
 'Y max (Tens. Ult. Stress (L-dir) with Temp. [MPa])',
 'Temperature [K]',
 'Time [hr]',
 'Other []',
 'Data Type Lab []',
 'Estimated point?']

## Populate the functional attribute
Add the test data to the functional attribute point-by-point, then view the attribute data. Column headers can be
omitted if they aren't required to represent the data.

In [11]:
for x, y in zip(x_values, y_values):
    func.add_point({'Temperature':x, 'y':y, 'Data Type Lab':'Test Data'})
func.value

[['Y min (Tens. Ult. Stress (L-dir) with Temp. [MPa])',
  'Y max (Tens. Ult. Stress (L-dir) with Temp. [MPa])',
  'Temperature [K]',
  'Time [hr]',
  'Other []',
  'Data Type Lab []',
  'Estimated point?'],
 [2078.31, 2078.31, 422.0389938964844, None, None, 'Test Data', False],
 [1734.37646484375,
  1734.37646484375,
  699.8172607421875,
  None,
  None,
  'Test Data',
  False],
 [2399.146240234375,
  2399.146240234375,
  194.2612762451172,
  None,
  None,
  'Test Data',
  False],
 [1263.0048828125,
  1263.0048828125,
  810.9284057617188,
  None,
  None,
  'Test Data',
  False],
 [2189.89, 2189.89, 294.2613938964844, None, None, 'Test Data', False],
 [1848.140014648438,
  1848.140014648438,
  588.7060546875,
  None,
  None,
  'Test Data',
  False]]

Then add the fitted data to the functional attribute point-by-point, and view the attribute data with series number as an extra column.

In [12]:
for x, y in zip(x_fit, y_fit):
    func.add_point({'Temperature':x, 'y':y, 'Data Type Lab':'Fitted Data'})
func.data_with_series_number

[['Y min (Tens. Ult. Stress (L-dir) with Temp. [MPa])',
  'Y max (Tens. Ult. Stress (L-dir) with Temp. [MPa])',
  'Temperature [K]',
  'Time [hr]',
  'Other []',
  'Data Type Lab []',
  'Estimated point?',
  'Series number'],
 [2078.31, 2078.31, 422.0389938964844, None, None, 'Test Data', False, 1],
 [1734.37646484375,
  1734.37646484375,
  699.8172607421875,
  None,
  None,
  'Test Data',
  False,
  1],
 [2399.146240234375,
  2399.146240234375,
  194.2612762451172,
  None,
  None,
  'Test Data',
  False,
  1],
 [1263.0048828125,
  1263.0048828125,
  810.9284057617188,
  None,
  None,
  'Test Data',
  False,
  1],
 [2189.89, 2189.89, 294.2613938964844, None, None, 'Test Data', False, 1],
 [1848.140014648438,
  1848.140014648438,
  588.7060546875,
  None,
  None,
  'Test Data',
  False,
  1],
 [2392.1796767662495,
  2392.1796767662495,
  194.2612762451172,
  None,
  None,
  'Fitted Data',
  False,
  2],
 [2333.081679240432,
  2333.081679240432,
  226.71744095651727,
  None,
  None,
  'F

Adjust the series linestyles (`series_linestyles` is a dictionary, indexed with integers).

In [13]:
func.series_linestyles[1] = 'Markers'
func.series_linestyles[2] = 'Lines'
func.series_linestyles

{1: 'Markers', 2: 'Lines'}

## Write your changes to MI
Set the attributes you've modified to update, and write the new record to the server.

In [14]:
record.set_attributes([func])
mi.update([record])

[<Record long name:STK Example 8:Tue May 10 22:44:28 2022>]